<hr>

## Data Loads for Vector Demo ##

### This should only need to be run once per project, as the schema gets stored in the repo ###

1. Run this **AFTER** Deploying the engine
2. Copy password from the engine deployment, use it in the input field when asked

**note** if the connection doesn't get read properly, it will ask for a username and host

Reads the connection information from the SQL Kernel userinfo.db to connect and create two foreign tables:
- web_comment_ft - retail customer comments
- glove_6B_50d_ft - GlovE 50d vector table



In [ ]:
import sqlite3
import teradatasql, json, getpass


# Create a SQL connection to our SQLite database
con = sqlite3.connect("/home/jovyan/JupyterLabRoot/Teradata/Config/userinfo.db")
cur = con.cursor()

# Return all results from the connection profile table
cur.execute('SELECT * FROM connectionprofile;')
res = cur.fetchall()

username = ''
host = ''
for ln in res:
    if '_private' in ln[0]: # we have the local connection context
        username = ln[3]
        host = ln[4]
if len(username) == 0: username = input('Please enter username: ')
if len(host) == 0: host = input('Please enter hostname or IP address: ')

print(username)
print(host)

con.close()


# bucket location for the data
bucket = 'vantagecloud-lake-demo-data'

In [ ]:


with teradatasql.connect(host = host, 
                     user = username, 
                     password = getpass.getpass(f'Password for user {username}: ')) as con:
    cur = con.cursor()
    try:
        cur.execute('DROP TABLE web_comment_ft;')
    except Exception as e:
        # Table already exists
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise
            
    qry = f'''
    CREATE MULTISET FOREIGN TABLE web_comment_ft
    
    (comment_id DECIMAL(18,0),
    customer_id DECIMAL(18,0),
    comment_text VARCHAR(50000) CHARACTER SET LATIN CASESPECIFIC,
    comment_summary VARCHAR(50000) CHARACTER SET LATIN CASESPECIFIC)
    
    USING
        (
          LOCATION  ('/s3/{bucket}.s3.amazonaws.com/DEMO_OFS/WEB_COMMENT/manifest.json')
          MANIFEST  ('TRUE')
          STOREDAS  ('PARQUET')
    );
    '''
    cur.execute(qry)
    
    qry = '''
    REPLACE VIEW web_commentV AS SELECT
    CAST(comment_id AS INTEGER) comment_id,
    CAST(customer_id AS INTEGER) customer_id,
    comment_text,
    comment_summary
    FROM web_comment_ft;'''
    
    cur.execute(qry)
    
    try:
        cur.execute('DROP TABLE glove_6B_50d_ft;')
    except Exception as e:
        # Table already exists
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise
    
        qry = f'''
CREATE MULTISET FOREIGN TABLE glove_6B_50d_ft
     (
      doc_id VARCHAR(50) CHARACTER SET LATIN NOT CASESPECIFIC,
      v1 FLOAT,
      v2 FLOAT,
      v3 FLOAT,
      v4 FLOAT,
      v5 FLOAT,
      v6 FLOAT,
      v7 FLOAT,
      v8 FLOAT,
      v9 FLOAT,
      v10 FLOAT,
      v11 FLOAT,
      v12 FLOAT,
      v13 FLOAT,
      v14 FLOAT,
      v15 FLOAT,
      v16 FLOAT,
      v17 FLOAT,
      v18 FLOAT,
      v19 FLOAT,
      v20 FLOAT,
      v21 FLOAT,
      v22 FLOAT,
      v23 FLOAT,
      v24 FLOAT,
      v25 FLOAT,
      v26 FLOAT,
      v27 FLOAT,
      v28 FLOAT,
      v29 FLOAT,
      v30 FLOAT,
      v31 FLOAT,
      v32 FLOAT,
      v33 FLOAT,
      v34 FLOAT,
      v35 FLOAT,
      v36 FLOAT,
      v37 FLOAT,
      v38 FLOAT,
      v39 FLOAT,
      v40 FLOAT,
      v41 FLOAT,
      v42 FLOAT,
      v43 FLOAT,
      v44 FLOAT,
      v45 FLOAT,
      v46 FLOAT,
      v47 FLOAT,
      v48 FLOAT,
      v49 FLOAT,
      v50 FLOAT)
    USING
        (
          LOCATION  ('/s3/{bucket}.s3.amazonaws.com/DEMO_OFS/GLOVE_6B_50D/manifest.json')
          MANIFEST  ('TRUE')
          STOREDAS  ('PARQUET')
    );
    '''
    cur.execute(qry)